In [2]:
import sys
import re
import cv2
import matplotlib.pyplot as plt
sys.path.append("../")
from lib.check import check_paths
paths = check_paths()

Check completed!


In [3]:
# # .jpeg to .pdf converter
# from PIL import Image
# import os

# image = Image.open(paths['data_path']/'Image.jpeg')
# pdf_path = paths['out_path']/'Image.pdf'
# image.save(pdf_path, 'PDF', resolution=100.0)

In [4]:
# from PyPDF2 import PdfReader
# pdf = PdfReader(paths['data_path']/'comprobante.pdf')
# text = ""
# for page in pdf.pages:
#     text += page.extract_text()
# if "CFE" in text:
#     try:
#         parts = text.split("Servicios Básicos")
#         if len(parts) > 1:
#             address_block = parts[1].strip()
#             lines = [line.strip() for line in address_block.split('\n') if line.strip()]
            
#             if len(lines) >= 2:
#                 line0 = lines[0]
#                 line1 = "".join(lines[1].split(",")[1:2]).strip()
#                 import re
#                 line1 = re.sub(r'[^\d]', '', line1)
#                 line2 = lines[2] if len(lines) > 2 else ""

#                 if ", colonia" in line0:
#                     street, colony = line0.split(", colonia")
#                     colony = "colonia" + colony
#                 else:
#                     street = line0
#                     colony = ""
                
#                 cp_line = line1
                
#                 city = line2.split(".")[0]
                
#                 domicile = [street.strip(), colony.strip(', '), cp_line.strip(', '), city.strip()]
#                 print("Found Domicile from PDF:")
#                 print(f"Street: {domicile[0]}")
#                 print(f"Colony: {domicile[1]}")
#                 print(f"CP Line: {domicile[2]}")
#                 print(f"City: {domicile[3]}")
#             else:
#                  print("Address lines not found in expected format.")
#     except Exception as e:
#         print(f"Error parsing address: {e}")


In [5]:
# from PIL import Image
# import pytesseract

# try:
#     image_path = paths['data_path'] / 'Image.jpeg'
#     image = Image.open(image_path)
#     text = pytesseract.image_to_string(image)
#     #print(text)

#     lines = [line.strip() for line in text.split('\n') if line.strip()]
#     domicile = []

#     for i, line in enumerate(lines):
#         if "C.P." in line:
#             if i >= 2:
#                 street = lines[i-2].split('|')[0].strip()
#                 colony = lines[i-1]
#                 cp_city = " ".join(line.split(' ')[1:2]).strip().rstrip(",")
                
#                 city = ""
#                 if i + 1 < len(lines) and "Estado de cuenta" not in lines[i+1]:
#                      city = lines[i+1]
#                 elif i + 2 < len(lines) and "Estado de cuenta" not in lines[i+2]:
#                      city = " ".join(lines[i+2].split(' ')[1:3])
                
#                 domicile = [street, colony, cp_city, city]
#                 print("Found Domicile:")
#                 print(f"Street: {street}")
#                 print(f"Colony: {colony}")
#                 print(f"CP Line: {cp_city}")
#                 print(f"City: {city}")
#                 break
    
#     if not domicile:
#         print("Could not find domicile (C.P. anchor).")

# except Exception as e:
#     print(f"Error: {e}")


In [6]:
# from structocr import StructOCR
# from dotenv import load_dotenv
# load_dotenv()
# client = StructOCR(os.getenv("STRUCT_OCR_API_KEY"))

# def scan_mexico_id():
#     image_path = paths["data_path"]/"ine zai.pdf"
#     # pdf to image
#     from pdf2image import convert_from_path
#     pages = convert_from_path(image_path)
#     for i, page in enumerate(pages):
#         page.save(paths["out_path"]/f"ine_{i}.jpeg", 'JPEG')
#     try:
#         print(f"Scanning {image_path}...")
        
#         result = client.scan_national_id(paths["out_path"]/f"ine_0.jpeg")

#         if result.get('success'):
#             data = result['data']
#             print("México Extraction Successful!")
            
#             print(f"Region:      {data.get('country_code')} (Series: {data.get('card_series')})")
#             print(f"Name:        {data.get('given_names')} {data.get('surname')}")
#             print(f"ID Number:   {data.get('document_number')}")
            
#             print(f"Personal #:  {data.get('personal_number')}")
            
#             print(f"DOB:         {data.get('date_of_birth')} ({data.get('sex')})")
#             print(f"Address:     {data.get('address')}")
#         else:
#             print(f"Extraction Failed: {result.get('error')}")

#     except Exception as e:
#         print(f"An error occurred: {e}")

# if __name__ == "__main__":
#     scan_mexico_id()

In [14]:
import re
import json
from surya.detection import DetectionPredictor
from surya.recognition import RecognitionPredictor
from PIL import Image

langs = ["es"]
det_predictor = DetectionPredictor()
rec_predictor = RecognitionPredictor()

try:
    from pdf2image import convert_from_path
    pages = convert_from_path(paths['data_path']/'ine zai.pdf')
    image_langs = [langs] * len(pages)
    predictions = rec_predictor(pages, image_langs, det_predictor=det_predictor)
except Exception as e:
    print(f"Error: {e}")

def parse_mrz_name(text):
    # Match the name line in MRZ (last line usually, format SURNAME<<NAME)
    # Example: FRANCO<BRAVO<<ZAIRA<DELFINA<<<
    mrz_name_match = re.search(r"([A-Z<]+<<[A-Z<]+)", text)
    if mrz_name_match:
        full_mrz = mrz_name_match.group(1)
        parts = full_mrz.split('<<')
        surnames = parts[0].replace('<', ' ').strip()
        given_names = parts[1].replace('<', ' ').strip()
        return f"{given_names} {surnames}".strip()
    return None

def extract_robust(text):
    data = {}
    text = re.sub(r'\n+', '\n', text)
    
    # 1. Name: Try MRZ first, then fall back to label
    mrz_name = parse_mrz_name(text)
    
    name_match = re.search(r"NOMBRE\s*(.*?)\s*DOMICILIO", text, re.DOTALL | re.IGNORECASE)
    potential_noisy_lines = []
    if name_match:
        name_block = name_match.group(1).strip()
        name_lines = name_block.split('\n')
        label_name_parts = []
        for l in name_lines:
            ls = l.strip()
            if len(ls) < 3: continue
            # If it looks like noise or address related, keep it for later
            if any(x in ls.lower() for x in ["apper", "copport", "credenzial", "votar", "electoral"]):
                 potential_noisy_lines.append(ls)
                 continue
            label_name_parts.append(ls)
        data["Nombre"] = mrz_name or " ".join(label_name_parts)
    
    # 2. Domicilio: Capture between labels and merge with noisy lines if they seem relevant
    addr_match = re.search(r"DOMICILIO\s*(.*?)\s*(?:CLAVE|CURP|ESTADO|REGISTRO|FECHA|SECCION|VIGENCIA)", text, re.DOTALL | re.IGNORECASE)
    if addr_match:
        addr_block = addr_match.group(1).strip()
        # Sometimes address lines are mis-detected in the Name section
        all_addr = potential_noisy_lines + addr_block.split('\n')
        # Deduplicate and clean
        clean_addr = []
        for l in all_addr:
            ls = l.strip()
            # Filter out obvious non-address noise like 'votar'
            if ls and not any(x in ls.lower() for x in ["votar", "electoral", "credenzial"]):
                if ls not in clean_addr:
                    clean_addr.append(ls)
        data["Domicilio"] = " ".join(clean_addr).replace('\n', ' ')

    # 3-7. Standard Fields
    clave_match = re.search(r"[A-Z]{6}\d{8}[A-Z]\d{3}", text)
    data["Clave de Elector"] = clave_match.group(0) if clave_match else None
    
    curp_match = re.search(r"[A-Z]{4}\d{6}[A-Z]{6}\d{2}", text)
    data["CURP"] = curp_match.group(0) if curp_match else None
    
    dob_match = re.search(r"(\d{2}/\d{2}/\d{4})", text)
    if dob_match:
        data["Fecha de Nacimiento"] = dob_match.group(1)
    elif data.get("CURP"):
        y, m, d = data["CURP"][4:6], data["CURP"][6:8], data["CURP"][8:10]
        century = "19" if int(y) > 30 else "20"
        data["Fecha de Nacimiento"] = f"{d}/{m}/{century}{y}"
    
    seccion_match = re.search(r"(?:SECCI.N|SECONON|SECCON|SECOION)\s*(\d+)", text, re.IGNORECASE)
    data["Sección"] = seccion_match.group(1) if seccion_match else None
    
    # 7. Vigencia
    vigencia_match = re.search(r"(\d{4}\s*-\s*\d{4})", text)
    data["Vigencia"] = vigencia_match.group(1) if vigencia_match else None
    
    return data

all_text = "\n".join([line.text for result in predictions for line in result.text_lines])
extracted_data = extract_robust(all_text)
print(json.dumps(extracted_data, indent=4, ensure_ascii=False))


Loaded detection model s3://text_detection/2025_02_28 on device mps with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device mps with dtype torch.float16


Recognizing Text: 100%|██████████| 1/1 [00:05<00:00,  5.52s/it]

{
    "Nombre": "FRANCO ZAIRA DELFINA",
    "Domicilio": "COLDEFENSORES DE LA REPUBLIC",
    "Clave de Elector": "FRBRZR94111021M900",
    "CURP": "FABZ941110MPLRRR03",
    "Fecha de Nacimiento": "10/11/1994",
    "Sección": null,
    "Vigencia": "2022-2032"
}
